In [ ]:
import numpy
import os
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import nn
from torch.nn import Sequential, Conv1d, MaxPool1d, Flatten, Linear,ReLU,Softmax,Tanh
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class MyData(Dataset):

    def __init__(self, root_dir, label_dir):
        self.root_dir = root_dir
        self.label_dir = label_dir
        self.path = os.path.join(self.root_dir, self.label_dir)
        self.data_path = os.listdir(self.path)

    def __getitem__(self, idx):
        txt_name = self.data_path[idx]
        data_item_path = os.path.join(self.root_dir, self.label_dir, txt_name)
        data = numpy.loadtxt(data_item_path)
        
        #list = []
        #data = list.append(data)
        
        label = self.label_dir
        return data, label

    def __len__(self):
        return len(self.data_path)

    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model1 = Sequential(
            Conv1d(1, 16, kernel_size=11),
            Conv1d(16, 16, kernel_size=3),
            Conv1d(16, 16, kernel_size=3),
            MaxPool1d(3),
            Conv1d(16, 64, kernel_size=3),
            Conv1d(64, 64,kernel_size=3),
            MaxPool1d(3),
            Conv1d(64, 64,kernel_size=3),
            Conv1d(64, 64,kernel_size=3),
            MaxPool1d(3),
            Flatten(),
            Linear(448, 64),
            Linear(64, 3)
        )

    def forward(self, x):
        x = self.model1(x)
        return x

In [ ]:
train_data_dir = 'E:\\1D-Data\\train\\'
test_data_dir = 'E:\\1D-Data\\val\\'
bus_label_dir = '0'
car_label_dir = '1'
man_label_dir = '2'

In [ ]:
train_bus_data = MyData(train_data_dir,bus_label_dir)
train_car_data = MyData(train_data_dir,car_label_dir)
train_man_data = MyData(train_data_dir,man_label_dir)
train_data = train_bus_data+train_car_data+train_man_data

In [ ]:
test_bus_data = MyData(test_data_dir,bus_label_dir)
test_car_data = MyData(test_data_dir,car_label_dir)
test_man_data = MyData(test_data_dir,man_label_dir)
test_data = test_bus_data+test_car_data+test_man_data

In [ ]:
BatchSize = 1

In [ ]:
Data_loader_train = torch.utils.data.DataLoader(train_data, batch_size=BatchSize, shuffle=True)  # 训练集
Data_loader_test = torch.utils.data.DataLoader(test_data, batch_size=BatchSize, shuffle=True)  # 测试集

In [ ]:
# 获取数据集长度
dataset_train_size = len(Data_loader_train)
dataset_test_size = len(Data_loader_test)
print("训练数据集长度：{}".format(dataset_train_size))
print("测试数据集长度：{}".format(dataset_test_size))

In [ ]:
OneD_nn = Net()

In [ ]:
# 损失函数
loss = nn.CrossEntropyLoss()

# 优化器
learn_rate = 1e-2
optimizer = torch.optim.SGD(OneD_nn.parameters(), lr=learn_rate)

# 设置训练网络参数
# 记录训练次数
total_train_step = 0
# 记录测试次数
total_test_step = 0
# 训练的轮数
epoch = 30
# 添加tensorboard
writer = SummaryWriter("log")

for i in range(epoch):
    print("----第 {} 轮训练开始----".format(i + 1))

    # 训练步骤开始
    OneD_nn.train()  # 有dropout和normlizer层需要调用
    for data in Data_loader_train:
        datas, targets = data
        empty_list_1 = []
        for i in range(BatchSize):
            #print(data_1[i].size())
            #print(data_1[i])
            b = datas[i].numpy()
            b=[b.tolist()]
            empty_list_1.append(b)
        empty_list_1 =torch.Tensor(empty_list_1)
        
        list = []
        for i in range(BatchSize):
            list.append(int(targets[i]))
        list = torch.LongTensor(list)  
        
        '''
        if targets == ('bus',):
            targets = [0]
        elif targets == ('car',):
            targets = [1]
        elif targets == ('man',):
            targets = [2] 
        '''
        #targets = torch.LongTensor(targets)
        #print(datas)
        output_x = OneD_nn(empty_list_1)
        #print(targets)
        #print(output_x)
        loss_x = loss(output_x, list)

        # 优化器优化模型
        optimizer.zero_grad()  # 梯度清0
        loss_x.backward()  # 求解grad梯度
        optimizer.step()  # 更新weight

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}，Loss：{} ".format(total_train_step, loss_x.item()))
            writer.add_scalar("train_loss", loss_x.item(), total_train_step)

    # 测试步骤开始
    OneD_nn.eval()  # 同理
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in Data_loader_test:
            datas, targets = data
            
            empty_list_2 = []
            for i in range(BatchSize):
                #print(data_1[i].size())
                #print(data_1[i])
                b = datas[i].numpy()
                b=[b.tolist()]
                empty_list_2.append(b)
            empty_list_2 =torch.Tensor(empty_list_2)
            
            list = []
            for i in range(BatchSize):
                list.append(int(targets[i]))
            list = torch.LongTensor(list) 
            '''
            if targets == ('bus',):
                targets = [0]
            elif targets == ('car',):
                targets = [1]
            elif targets == ('man',):
                targets = [2] 
            ''' 
            #targets = torch.LongTensor(targets)
            
            output_x = OneD_nn(empty_list_2)
            loss_x = loss(output_x, list)
            total_test_loss = total_test_loss + loss_x.item()
            accuracy = (output_x.argmax(1) == list).sum()
            total_accuracy = total_accuracy + accuracy
    print("整体测试集上的loss：{}".format(total_test_loss))
    print("整体测试集上的正确率：{}".format(total_accuracy / dataset_test_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy / dataset_test_size, total_test_step)
    total_test_step = total_test_step + 1
    if i == epoch-1:
        torch.save(OneD_nn, "OneD_nn_{}.pth".format(i))
    # torch.save(xjl_nn.state_dict(),"xjl_nn_{}.pth".format(i))
    print("模型已保存")

writer.close()
